# Intelligent Mental Health AI chatbot Assistant

In [ ]:
!pip install langchain pypdf faiss-cpu openai tiktoken -U langchain-community datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 74.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.

In [ ]:
from langchain.chains import LLMChain
import os
import openai
from langchain.prompts import PromptTemplate
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings
# from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

# from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager

from langchain.agents import initialize_agent, Tool

from langchain.chains import LLMMathChain
from langchain.utilities import GoogleSerperAPIWrapper

Upload the pdf file and set file path

In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from tqdm import tqdm

# Specify the directory containing the PDF files
pdf_directory = '/content/data'
# Create an empty list to store the documents
all_documents = []

# Iterate over all files in the directory
for filename in tqdm(os.listdir(pdf_directory)):
    # Check if the file is a PDF
    if filename.endswith('.pdf'):
        # Construct the full file path
        file_path = os.path.join(pdf_directory, filename)

        # Load the PDF using PyPDFLoader
        loader = PyPDFLoader(file_path=file_path)
        documents = loader.load()

        # Split the documents into smaller chunks (optional)
        text_splitter = RecursiveCharacterTextSplitter()
        documents = text_splitter.split_documents(documents)
        for doc in documents:
            doc.page_content = doc.page_content.replace('\n', ' ')
        # Append the documents to the list
        all_documents.extend(documents)

# Now you have a list of all the documents from all the PDF files
print(f"Total documents loaded: {len(all_documents)}")

100%|██████████| 2/2 [00:33<00:00, 16.90s/it]

Total documents loaded: 1523


In [ ]:
all_documents[30]

Document(metadata={'source': '/content/data/The Encyclopedia of Mental Health (Facts on File Library of Health and Living) - PDF Room.pdf', 'page': 23}, page_content='ment of infant girls and overcrowded orphanages,  factors in the government’s decision to facilitate  international adoptions. Adoptions from Russia have grown significantly  since the Soviet Union dissolved. In 2001 13 times as  many children were adopted from Russia as in 1992,  when it was declared independent. Guatemala leads  Central and South America in adoptions. Transracial adoption refers to children who are  placed with an adoptive family of another race or  ethnicity. The new families face unique cultural  issues. An estimated 15 percent of the 36,000 adop- tions from foster care in 1998 were transracial or  transcultural. National Adoption Information Clearinghouse  (NAIC) The National Adoption Information Clearinghouse  (NAIC) was established by Congress in 1987 to  provide free information on all aspects of 

In [ ]:
# from datasets import load_dataset

# ds1 = load_dataset("Amod/mental_health_counseling_conversations")

# ds2 = load_dataset("heliosbrahma/mental_health_chatbot_dataset")
# ds3 = load_dataset("mpingale/mental-health-chat-dataset")

In [ ]:
import os
os.environ['OPENAI_API_KEY'] ='sk-xxxx'
os.environ['SERP_API_KEY']='yyyy'

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(show_progress_bar=True)

vector1 = embeddings.embed_query('I am helpful chatbot')

len(vector1)

  0%|          | 0/1 [00:00<?, ?it/s]

1536

In [ ]:
from langchain.vectorstores import FAISS

index = FAISS.from_documents(all_documents, embeddings)

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
index.save_local('/content/vector_faiss')

In [ ]:
INDEX_PATH='/content/vector_faiss'
index=FAISS.load_local(INDEX_PATH, embeddings, allow_dangerous_deserialization=True)

In [ ]:
index.similarity_search_with_relevance_scores(
    "mental health"
)

  0%|          | 0/1 [00:00<?, ?it/s]

[(Document(metadata={'source': '/content/data/The Encyclopedia of Mental Health (Facts on File Library of Health and Living) - PDF Room.pdf', 'page': 4}, page_content='iv M ental health is a continuum, ranging from  the annoyances of stressful random nui- sances to serious mental disorders. W e hope that  readers will find this third edition of The Encyclo- pedia of Mental Health helpful while coping with  everyday challenges in healthy ways. According to Healthy People 2010,  published by  the U.S. government, mental health is a state of  successful mental functioning, resulting in pro- ductive activities, fulfilling relationships, and the  ability to adapt to change and cope with adversity.  This blueprint for good health indicates that men- tal health is indispensable to well-being, family  and interpersonal relationships, and one’s contri- bution to society. Much has changed in the world since the sec- ond edition of T h e  E n c y c l o p e d i a  o f  M e n t a l  H e a l t h  ap

In [ ]:
llm = ChatOpenAI()

prompt_template = """Answer the question as professional mental health Counselor
Context: {context}

{chat_history}
Human: {question}
Assistant:"""

# Create prompt from prompt template
prompt = PromptTemplate(
    input_variables=["context", "question", "chat_history"],
    template=prompt_template,
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)


memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)
handler = StdOutCallbackHandler()
llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
    verbose=True,
    memory=memory
)

retriever = index.as_retriever(
search_type="similarity",
search_kwargs={'k': 10}
    )

<ipython-input-20-8b7b4bf93599>:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI()
<ipython-input-20-8b7b4bf93599>:18: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)
<ipython-input-20-8b7b4bf93599>:20: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(


In [ ]:
from langchain.utilities import GoogleSerperAPIWrapper

# Outputs

In [ ]:
question = "what are the common symptoms of anxiety"
docs = retriever.get_relevant_documents(question)
res=llm_chain.run(question=question, context=docs, callbacks=[handler])
res

<ipython-input-22-ed5da1329af2>:2: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  docs = retriever.get_relevant_documents(question)


  0%|          | 0/1 [00:00<?, ?it/s]

<ipython-input-22-ed5da1329af2>:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  res=llm_chain.run(question=question, context=docs, callbacks=[handler])




> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional mental health Counselor
Context: [Document(metadata={'source': '/content/data/The Encyclopedia of Mental Health (Facts on File Library of Health and Living) - PDF Room.pdf', 'page': 437}, page_content='Stress Signals Nervous tic Muscular aches Inability to sleep Increased sweating Stuttering Nausea or stomach pain Grinding teeth Headache, dizziness Low-grade infections Rash or acne Desire to cry or crying Constipation or diarrhea Frigidity or impotence High blood pressure Dry mouth or throat Irritability or bad temper Lethargy or inability to work Cold, clammy or clenched hands Sudden bursts of energy Finger-tapping Depression Fear, panic or anxiety Hives Coughing Nagging Fatigue Pacing Frowning Restlessness Accident prone Common Thoughts and Feelings Impulsive Freeze up Become rigid Falling apart Thoughts “jumble up” Feeling tense Constant worrying Feeling time pressure World is caving in S

"As a professional mental health counselor, I can provide information on the common symptoms of anxiety. Anxiety can manifest in various physical and psychological symptoms, including nervousness, muscle tension, inability to sleep, increased sweating, headache, dizziness, irritability, restlessness, and feelings of impending doom. It can also lead to symptoms such as fatigue, stomach pain, trembling, and difficulty concentrating. It's important to note that anxiety symptoms can vary from person to person, and seeking professional help is crucial for proper diagnosis and treatment."

In [ ]:
question = "What is ADHD"
docs = retriever.get_relevant_documents(question)
res=llm_chain.run(question=question, context=docs, callbacks=[handler])
res

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional mental health Counselor
Context: [Document(metadata={'source': '/content/data/The Encyclopedia of Mental Health (Facts on File Library of Health and Living) - PDF Room.pdf', 'page': 160}, page_content='with the assistance of a trusted expert diagnostic  team. Parents should locate mental health services  in their area, National Institute of Mental Health  (NIMH) clinical trials, and listings of professionals  and organizations. Attention-Deﬁ cit/Hyperactivity Disorder (ADHD) Attention-Deficit/Hyperactivity Disorder (ADHD)  is one of the most common mental disorders that  152    detoxification 001-476_mentalhealth_a-z.indd   152 9/27/07   12:34:15 PM'), Document(metadata={'source': '/content/data/The Encyclopedia of Mental Health (Facts on File Library of Health and Living) - PDF Room.pdf', 'page': 161}, page_content='develop in children. Children with ADHD have  impaired functioning in many 

'ADHD, or Attention-Deficit/Hyperactivity Disorder, is one of the most common mental disorders that develop in children. Children with ADHD have impaired functioning in various settings, such as at home, school, and in relationships with peers. The condition is characterized by a pattern of inattention and/or hyperactivity-impulsivity that is more frequent and severe than what is typically observed in individuals at comparable developmental levels. Symptoms of ADHD include impulsiveness, hyperactivity, and inattention, which can impact social, academic, and occupational functioning. Diagnosis usually occurs before age seven, with evidence of interference with developmentally appropriate functioning. Early diagnosis and intervention are important to prevent negative attitudes towards school and behavioral issues in later life. Treatment approaches for ADHD may include behavior management, cognitive therapy, medication, and educational interventions.'

# Code For Chainlit (Interface)
To execute the interface we have a app.py file which can be executed from terminal.

In [ ]:
from langchain.chains import LLMChain
import chainlit as cl
import os
import openai
from langchain.prompts import PromptTemplate
# from langchain.vectorstores import FAISS
from langchain_community.vectorstores import FAISS
#from langchain.embeddings.openai import OpenAIEmbeddings
from langchain_community.embeddings import OpenAIEmbeddings
# from langchain.memory import ConversationBufferMemory
from langchain.memory import ConversationBufferWindowMemory

# from langchain.chains import RetrievalQA
from langchain_community.chat_models import ChatOpenAI
from langchain.callbacks import StdOutCallbackHandler
from langchain.prompts import PromptTemplate
from langchain.callbacks.manager import CallbackManager

from langchain.agents import initialize_agent, Tool

from langchain.chains import LLMMathChain
from langchain_community.utilities import GoogleSerperAPIWrapper


openai_api_key = "xxxx"
serper_api_key="yyyy"


VECTOR_STORE_PATH = "./vector_faiss/"

openai.api_key = openai_api_key
os.environ["SERPER_API_KEY"]=serper_api_key


@cl.cache
def instantiate():

    llm = ChatOpenAI()


    embeddings = OpenAIEmbeddings(show_progress_bar=True)

    # Load vector store
    # index=FAISS.load_local('https://drive.google.com/file/d/1-DgmC8rYns-IRmIGZZF1TcaHztX2i7vq/view?usp=sharing', embeddings, allow_dangerous_deserialization=True)
    index=FAISS.load_local(VECTOR_STORE_PATH, embeddings,allow_dangerous_deserialization=True)
    return llm , index

llm , vector_store= instantiate()

llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
# Set up tools

search = GoogleSerperAPIWrapper()



tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]


# Set up agent
agent = initialize_agent(
    tools,
    llm,
    agent="conversational-react-description",
    verbose=True,
    memory=ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5),
    return_intermediate_steps=True,
)


# Set up conversation chain
@cl.on_chat_start
def main():
    cl.user_session.set(
        "message_history",
        [{"role": "system", "content": "You are an empathetic mental health counselor focused on providing support and guidance."}],
    )


    prompt_template = """"Answer the question as a supportive and empathetic mental health counselor. Always maintain professional boundaries and encourage seeking professional help when appropriate.
    Context: {context}


    {chat_history}
    Human: {question}
    Assistant:"""

    # Create prompt from prompt template
    prompt = PromptTemplate(
        input_variables=["context", "question", "chat_history"],
        template=prompt_template,
        # partial_variables={"format_instructions": parser.get_format_instructions()},
    )


    memory = ConversationBufferWindowMemory(memory_key="chat_history", input_key="question", k=5)
    handler = StdOutCallbackHandler()
    llm_chain = LLMChain(
        llm=llm,
        prompt=prompt,
        verbose=True,
        memory=memory
    )

    cl.user_session.set("llm_chain", llm_chain)

    retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={'k': 10}
        )

    cl.user_session.set("retriever", retriever)
    cl.user_session.set("agent", agent)


@cl.on_message
async def main(message: cl.Message):
    llm_chain = cl.user_session.get("llm_chain")
    retriever= cl.user_session.get("retriever")
    agent = cl.user_session.get("agent")

    # Get the user's message
    user_input = message.content
    docs = retriever.get_relevant_documents(user_input)
    msg = cl.Message(content="")
    await msg.send()
    # Generate response using LLMChain
    response = await llm_chain.arun(question=user_input, context=docs, callbacks=[cl.LangchainCallbackHandler(stream_final_answer=True)],agent=agent)


    for part in (response):
        if token := part or "":
            await msg.stream_token(token)
    # # Send the response back to the user
    # await message.respond(response)
    await msg.update()

ModuleNotFoundError: No module named 'chainlit'

# Running the chatbot and Evaluating the output responses

In [ ]:
# Save the dataset as a JSON file for evaluation purposes
import json

# Mental health evaluation dataset

mental_health_dataset =[
    {
        "input_query": "What are the symptoms of anxiety?",
        "expected_response": "Physical symptoms of anxiety disorders may include nausea, dizziness, rapid heartbeat, and a feeling of impending doom. Psychological symptoms often involve excessive worry, irritability, and difficulty concentrating&#8203;:contentReference[oaicite:0]{index=0}."
    },
    {
        "input_query": "How can I manage stress effectively?",
        "expected_response": "Stress management includes maintaining a well-balanced diet, regular physical exercise, relaxation techniques such as deep breathing or meditation, and seeking professional help when necessary. Creating a low-stress environment is also essential&#8203;:contentReference[oaicite:1]{index=1}."
    },
    {
        "input_query": "What is the difference between sadness and depression?",
        "expected_response": "Sadness is a temporary emotional state, while depression is a clinical mood disorder characterized by symptoms like persistent feelings of hopelessness, changes in sleep or appetite, and an inability to concentrate&#8203;:contentReference[oaicite:2]{index=2}."
    },
    {
        "input_query": "What are common treatments for depression?",
        "expected_response": "Treatments for depression include cognitive-behavioral therapy, medications such as antidepressants, and in severe cases, electroconvulsive therapy (ECT). Therapy focuses on addressing negative thought patterns and improving coping mechanisms&#8203;:contentReference[oaicite:3]{index=3}&#8203;:contentReference[oaicite:4]{index=4}."
    },
    {
        "input_query": "How can I help a friend dealing with mental health issues?",
        "expected_response": "Support includes listening without judgment, showing empathy, encouraging professional help, and creating a safe space for open communication&#8203;:contentReference[oaicite:5]{index=5}."
    },
    {
        "input_query": "What are the warning signs of burnout?",
        "expected_response": "Signs of burnout include chronic fatigue, irritability, reduced performance, detachment from activities, and physical symptoms such as headaches or muscle tension&#8203;:contentReference[oaicite:6]{index=6}."
    },
    {
        "input_query": "Can physical exercise improve mental health?",
        "expected_response": "Regular physical exercise helps improve mental health by reducing stress, alleviating symptoms of anxiety and depression, and improving mood through endorphin release&#8203;:contentReference[oaicite:7]{index=7}."
    },
    {
        "input_query": "What is cognitive-behavioral therapy?",
        "expected_response": "Cognitive-behavioral therapy (CBT) is a form of psychotherapy that addresses dysfunctional emotions, behaviors, and thought processes through goal-oriented, systematic procedures&#8203;:contentReference[oaicite:8]{index=8}."
    },
    {
        "input_query": "Are there any apps for mental health support?",
        "expected_response": "Mental health apps, such as those for guided meditation or cognitive-behavioral strategies, can support individuals in managing anxiety and stress through accessible, user-friendly interfaces&#8203;:contentReference[oaicite:9]{index=9}&#8203;:contentReference[oaicite:10]{index=10}."
    },
    {
        "input_query": "What is mindfulness meditation?",
        "expected_response": "Mindfulness meditation involves focusing on the present moment in a non-judgmental way, often through breathing exercises or guided sessions. It can reduce stress and promote overall well-being&#8203;:contentReference[oaicite:11]{index=11}."
    },
    {
        "input_query": "How can I improve my sleep habits?",
        "expected_response": "Improving sleep involves maintaining a consistent sleep schedule, limiting caffeine and screen time before bed, and creating a comfortable sleeping environment&#8203;:contentReference[oaicite:12]{index=12}."
    },
    {
        "input_query": "What should I do during a panic attack?",
        "expected_response": "During a panic attack, slow, deep breathing, grounding techniques like focusing on physical sensations, and reminding oneself that the attack will pass can be helpful. Support from a trusted companion can also provide comfort&#8203;:contentReference[oaicite:13]{index=13}."
    },
    {
        "input_query": "What is post-traumatic stress disorder (PTSD)?",
        "expected_response": "PTSD is a condition triggered by a traumatic event, characterized by flashbacks, nightmares, heightened alertness, and emotional numbness. It often requires therapy and, in some cases, medication for management&#8203;:contentReference[oaicite:14]{index=14}."
    },
    {
        "input_query": "What are the benefits of journaling for mental health?",
        "expected_response": "Journaling can help reduce stress, clarify thoughts, and process emotions. It encourages self-reflection and personal growth&#8203;:contentReference[oaicite:15]{index=15}."
    },
    {
        "input_query": "How does social media affect mental health?",
        "expected_response": "Social media has both positive and negative effects on mental health. While it can foster connections, excessive use may lead to anxiety, depression, and low self-esteem&#8203;:contentReference[oaicite:16]{index=16}."
    }
]


# Save as JSON
output_file_path = '/content/sample_data/data/mental_health_evaluation_dataset.json'
with open(output_file_path, 'w') as file:
    json.dump(mental_health_dataset, file, indent=4)

output_file_path


'/content/sample_data/data/mental_health_evaluation_dataset.json'

In [ ]:
response_log = []

def log_response(input_query, chatbot_response):
    response_log.append({"input_query": input_query, "response": chatbot_response})


In [ ]:
for log in mental_health_dataset:
    question = log['input_query']
    docs = retriever.get_relevant_documents(question)
    res=llm_chain.run(question=question, context=docs, callbacks=[handler])
    log_response(question, res)

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional health Counselor
Context: [Document(metadata={'source': '/content/6.pdf.pdf', 'page': 437}, page_content='Stress Signals Nervous tic Muscular aches Inability to sleep Increased sweating Stuttering Nausea or stomach pain Grinding teeth Headache, dizziness Low-grade infections Rash or acne Desire to cry or crying Constipation or diarrhea Frigidity or impotence High blood pressure Dry mouth or throat Irritability or bad temper Lethargy or inability to work Cold, clammy or clenched hands Sudden bursts of energy Finger-tapping Depression Fear, panic or anxiety Hives Coughing Nagging Fatigue Pacing Frowning Restlessness Accident prone Common Thoughts and Feelings Impulsive Freeze up Become rigid Falling apart Thoughts “jumble up” Feeling tense Constant worrying Feeling time pressure World is caving in Stressors (in Your Family) Holidays, vacations Marital difﬁ culties Injury or illness Problems wi

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional health Counselor
Context: [Document(metadata={'source': '/content/6.pdf.pdf', 'page': 435}, page_content='dent on parents. As life is a series of progressions  through emotional stages, it is helpful to remem- ber that change and growth always involve some  degree of stress. In a family, several people are try- ing to cope with their own stress and the stress of  others about whom they care. Diet and exercise can help relieve stress. Nor- mal eating of three meals a day reduces effects of  stress for some people. “Crash diets” and “fad diets”  can lead to anxiety, depression and an inability to  maintain a good weight. Well-balanced meals pro- vide a slow release of necessary nutrients through- out the day. For some people, too much caffeine  causes additional stress by bringing on symptoms  of anxiety. Many people find that regular physical workouts  involving running, walking or exercising

  0%|          | 0/1 [00:00<?, ?it/s]

...



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional health Counselor
Context: [Document(metadata={'source': '/content/6.pdf.pdf', 'page': 357}, page_content='children report that their loved one does not com- municate, show affection or share in family life. The  therapist can help members of the family recognize  and cope with the range of emotions they feel, and  when needed, help them improve their communi- cation skills and learn techniques for parenting and  stress management. A newer technique for PTSD involves “rap”  groups, in which survivors of similar traumatic events  are encouraged to share their experiences and reac- tions. Group members help each other realize that  many people have gone through and experienced  similar events and emotions. Over time, the mem- bers gain an improved self-image and self-esteem.  Antidepressant medications have also been reported  to reverse symptoms of PTSD. Selective serotonin  reuptake inhibitor

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional health Counselor
Context: [Document(metadata={'source': '/content/6.pdf.pdf', 'page': 266}, page_content='self- understanding and serves as a way to record  progress but also gives the individual something she  can refer to when she wants a drink or a cigarette or  is about to give in to a desire to overeat. Journal keeping is used by many support groups  for overeaters, as well as those who wish to stop  smoking or drinking. Writing in a journal or diary  also serves the obvious purposes of recording the  events and impressions of the day and of improving  writing style. See also CONTROL; EATING DISORDERS; SUPPORT  GROUPS. Adams, Kathleen. Journal to the Self. New York: Warner  Books, 1990. 258    job security 001-476_mentalhealth_a-z.indd   258 9/27/07   12:34:32 PM'), Document(metadata={'source': '/content/6.pdf.pdf', 'page': 379}, page_content='tive rewards (positive reinforcement). For 

  0%|          | 0/1 [00:00<?, ?it/s]



> Entering new LLMChain chain...
Prompt after formatting:
Answer the question as professional health Counselor
Context: [Document(metadata={'source': '/content/6.pdf.pdf', 'page': 204}, page_content='fidence. People want to feel that they can rely on  their friends and that their friends will be open and  honest with them. Friendships are involved in maintaining good  mental health. Friendships can be supportive in our  daily lives as well as during periods of turmoil or  crisis. Individuals who experience depression often  report a lack of friends, although having a wide cir- cle of friends is not a preventive factor for depres- sion. Some reports have indicated that individuals  who have many social contacts may be healthier  and actually live longer than those who do not. Friendships also affect mental health because  they may challenge or be challenged by other rela- tionships in contemporary life. For example, an  employer, supervisor or teacher, particularly one  with an author

In [ ]:
len(response_log)

15

In [ ]:
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import jaccard_score

# BLEU Score for response similarity
def evaluate_responses_with_bleu(ground_truth, predictions):
    bleu_scores = [sentence_bleu([gt.split()], pred.split()) for gt, pred in zip(ground_truth, predictions)]
    avg_bleu = sum(bleu_scores) / len(bleu_scores)
    print(f"Average BLEU Score: {avg_bleu:.2f}")
    return {"BLEU Score": avg_bleu}

# Ground truth and predictions
ground_truth = [log['expected_response'] for log in mental_health_dataset]
predictions = [log['response'] for log in response_log]  # Ensure response_log is populated

# Evaluate
bleu_metrics = evaluate_responses_with_bleu(ground_truth, predictions)

print("BLEU Score:", bleu_metrics)


Average BLEU Score: 0.01
BLEU Score: {'BLEU Score': 0.007784141982354522}


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# Load a pre-trained embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')

def evaluate_responses_with_semantic_similarity(ground_truth, predictions):
    # Compute embeddings
    ground_truth_embeddings = model.encode(ground_truth)
    predictions_embeddings = model.encode(predictions)

    # Compute cosine similarity
    similarities = cosine_similarity(ground_truth_embeddings, predictions_embeddings)
    avg_similarity = similarities.diagonal().mean()  # Diagonal gives one-to-one comparison
    print(f"Average Semantic Similarity: {avg_similarity:.2f}")
    return {"Semantic Similarity": avg_similarity}

semantic_metrics = evaluate_responses_with_semantic_similarity(ground_truth, predictions)
print("Semantic Similarity:", semantic_metrics)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Average Semantic Similarity: 0.75
Semantic Similarity: {'Semantic Similarity': 0.74856085}
